In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [2]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-17 19:50:10,157 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-17 19:50:10,158 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-17 19:50:13,496 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:ca6ccd63f270d603ffa9bdf5a7ced184d6339796
2017-06-17 19:50:13,856 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-17 19:50:17,448 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-17 19:50:17,449 [INFO] [holmes.experiment.staging] Commit staged
2017-06-17 19:50:17,450 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-17 19:50:17,509 [INFO] [holmes.experiment.staging] Extracting default config from package


In [3]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [9,10,11]
hungarian_thresholds = [15]
minimum_detections = [30]

In [4]:
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

12 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp2u1rxpu6/M00-ChvUtd-000-09-15-30.json


In [5]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-1-3", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-1-3",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-17 19:51:22,883 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3


In [6]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [7]:
experiment.stage_experiment()

2017-06-17 19:51:31,277 [INFO] [holmes.experiment.base] staging measurement M00-ChvUtd-000-09-15-30
2017-06-17 19:51:31,759 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp2u1rxpu6/M00-ChvUtd-000-09-15-30.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/M00-ChvUtd-000-09-15-30.json
2017-06-17 19:51:32,201 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp2u1rxpu6/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz
2017-06-17 19:51:33,516 [INFO] [holmes.experiment.base] staging measurement M01-PS0304-001-09-15-30
2017-06-17 19:51:34,071 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmp2u1rxpu6/M01-PS0304-001-09-15-30.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps

{'experiment_id': 'PTA-95-LT-HT-DET-1-3',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/M00-ChvUtd-000-09-15-30.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-3',
   'job_id': 'PTA-95-LT-HT-DET-1-3-M00-ChvUtd-000-09-15-30',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-ChvUtd-000-09-15-30',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz'},
  {'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/M01-PS0304-001-09-15-30.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-3',
   'job_id': 'PTA-95-LT-HT-DET-1-3-M01-PS0304-001-09-15-30',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M01-PS0304-001-09-15-30',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/ca6ccd63f270d603ffa9bdf5a7ced184d63397

In [8]:
experiment.launch_experiment()

2017-06-17 19:51:50,260 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M00-ChvUtd-000-09-15-30
2017-06-17 19:51:50,646 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M01-PS0304-001-09-15-30
2017-06-17 19:51:50,984 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M02-PP0214-000-09-15-30
2017-06-17 19:51:51,258 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M03-ER0429-001-09-15-30
2017-06-17 19:51:51,528 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M04-ChvUtd-000-10-15-30
2017-06-17 19:51:51,775 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M05-PS0304-001-10-15-30
2017-06-17 19:51:52,176 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M06-PP0214-000-10-15-30
2017-06-17 19:51:52,499 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M07-ER0429-001-10-15-30
2017-06-17 19:51:52,764 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-1-3-M08-ChvUtd-000-11-15-30
2

{'experiment_id': 'PTA-95-LT-HT-DET-1-3',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-1-3/deps/M00-ChvUtd-000-09-15-30.json',
   'experiment_id': 'PTA-95-LT-HT-DET-1-3',
   'job_id': 'PTA-95-LT-HT-DET-1-3-M00-ChvUtd-000-09-15-30',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Sun, 18 Jun 2017 00:49:37 GMT',
      'via': '1.1 e8a1bacdefcba30a794b8f2b36714285.cloudfront.net (CloudFront)',
      'x-amz-cf-id': 'mlInhH4Wj-Fx7ZmdQUzZ6SKQU-0ls3MFHkcvFFquZMEEHBtEKHEKgw==',
      'x-amzn-requestid': '0120ea47-53c0-11e7-9372-7f75fc21fd25',
      'x-amzn-trace-id': 'sampled=0;root=1-5945ce21-27a69e5eac11b11b3c1ef67c',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': '0120ea47-53c0-11e7-9372-7f75fc21fd25',


In [10]:
import pandas as pd

In [11]:
data = pd.DataFrame({'mota':[.659,.656,.653,.791,.791,.790,.860,.859,.858,.765,.764,.763],
              'lt':[9,10,11,9,10,11,9,10,11,9,10,11],
              'ht':[15,15,15,15,15,15,15,15,15,15,15,15],
              'det':[30,30,30,30,30,30,30,30,30,30,30,30],
             'game':['ER0429','ER0429','ER0429','ChvUtd','ChvUtd','ChvUtd','PP0214','PP0214','PP0214','PS0304',
                    'PS0304','PS0304']})

In [13]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [14]:
output_notebook()

Loading BokehJS ...

In [15]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections
ht lt det                                      
15 9  30   0.76875     9         15          30
   10 30   0.76750    10         15          30
   11 30   0.76600    11         15          30

In [19]:
scat_lt_ht_det_1 = Scatter(data_lt_ht_det, x='lost', y='mota', color='detections', marker='hungarian')
show(scat_lt_ht_det_1)